## Using itk-elastix to register a single section to a BrainGlobe Atlas

BrainGlobe 2D image registration has not yet been released, so we're going to use the [`itk-elastix`](https://github.com/InsightSoftwareConsortium/ITKElastix) library directly.

In [1]:
# Download and load a 2D coronal image
import pooch
import tifffile
from pathlib import Path

# Use pooch to fetch data if it hasn't already been downloaded
image_url = "https://gin.g-node.org/neuroinformatics/image-analysis-courses/raw/master/misc/coronal_section.tif"
image_path = pooch.retrieve(image_url, path=Path.cwd().parent, fname="coronal_section.tif", known_hash=None, progressbar=True)
fixed_image = tifffile.imread(image_path)

In [2]:
# Load image into napari
import napari
viewer = napari.Viewer()
viewer.add_image(fixed_image)

<Image layer 'fixed_image' at 0x7fce6ca726b0>

In [3]:
# Instantiate a BrainGlobe Atlas API atlas class and choose an appropriate plane
from bg_atlasapi import BrainGlobeAtlas
atlas_plane = 230 
atlas = BrainGlobeAtlas("allen_mouse_25um")
moving_image = atlas.reference[atlas_plane]
viewer.add_image(moving_image)

<Image layer 'moving_image' at 0x7fce48a292a0>

In [4]:
# Resample so the images are the same scale
from scipy.ndimage import zoom
image_pixel_size = 5 # um
atlas_pixel_size = atlas.resolution[0]
downsampled_fixed_image = zoom(fixed_image, image_pixel_size / atlas_pixel_size)
viewer.add_image(downsampled_fixed_image)

<Image layer 'downsampled_fixed_image' at 0x7fce02b13a60>

In [5]:
# Define a registration function (see `itk-elastix` examples for more)
import itk
import numpy as np

def run_registration(
    fixed_image,
    moving_image,
    rigid=True,
    affine=True,
    bspline=True,
    affine_iterations="2048",
    log=False,
):
    # convert to ITK, view only
    fixed_image = itk.GetImageViewFromArray(fixed_image).astype(itk.F)
    moving_image = itk.GetImageViewFromArray(moving_image).astype(itk.F)

    # This syntax needed for 3D images. Leaving here for compatibility.
    elastix_object = itk.ElastixRegistrationMethod.New(
        fixed_image, moving_image
    )

    parameter_object = setup_parameter_object(
        rigid=rigid,
        affine=affine,
        bspline=bspline,
        affine_iterations=affine_iterations,
    )
    elastix_object.SetParameterObject(parameter_object)
    elastix_object.SetLogToConsole(log)

    # update filter object
    elastix_object.UpdateLargestPossibleRegion()

    # get results
    result_image = elastix_object.GetOutput()
    result_transform_parameters = elastix_object.GetTransformParameterObject()
    return np.asarray(result_image), result_transform_parameters


def setup_parameter_object(
    rigid=True,
    affine=True,
    bspline=True,
    affine_iterations="2048",
):
    parameter_object = itk.ParameterObject.New()

    if rigid:
        parameter_map_rigid = parameter_object.GetDefaultParameterMap("rigid")
        parameter_object.AddParameterMap(parameter_map_rigid)

    if affine:
        parameter_map_affine = parameter_object.GetDefaultParameterMap(
            "affine"
        )
        parameter_map_affine["MaximumNumberOfIterations"] = [affine_iterations]
        parameter_object.AddParameterMap(parameter_map_affine)

    if bspline:
        parameter_map_bspline = parameter_object.GetDefaultParameterMap(
            "bspline"
        )
        parameter_object.AddParameterMap(parameter_map_bspline)

    return parameter_object

In [6]:
# Register the atlas image to the sample
registered_image, result_transform_parameters = run_registration(downsampled_fixed_image, moving_image)

In [7]:
# Visualise the result
viewer.add_image(registered_image)

<Image layer 'registered_image' at 0x7fcd036661d0>

In [8]:
# Define a function to transform any other images from atlas to sample space
def transform_additional_image(moving_image, result_transform_parameters, mask=False):
    result_transform_parameters.SetParameter('FinalBSplineInterpolationOrder','0')
    
    # convert to ITK, view only
    moving_image = itk.GetImageViewFromArray(moving_image).astype(itk.F)
    
    # Load Transformix Object
    transformix_object = itk.TransformixFilter.New(moving_image)
    transformix_object.SetTransformParameterObject(result_transform_parameters)

    
    # Update object (required)
    transformix_object.UpdateLargestPossibleRegion()
    
    # Results of Transformation
    result_image = transformix_object.GetOutput()

    if mask:
        return np.asarray(result_image, dtype=int)
    else:
        return np.asarray(result_image)

In [9]:
# Apply transformation to the annotations image
warped_annotations = transform_additional_image(atlas.annotation[atlas_plane], result_transform_parameters, mask=True)
viewer.add_labels(warped_annotations)

<Labels layer 'warped_annotations' at 0x7fce025e67d0>

In [10]:
# Pick a value from atlas to inspect
from pprint import pprint
pprint(atlas.structures[672])

{'acronym': 'CP',
 'id': 672,
 'mesh': None,
 'mesh_filename': PosixPath('/home/adam/.brainglobe/allen_mouse_25um_v1.2/meshes/672.obj'),
 'name': 'Caudoputamen',
 'rgb_triplet': [152, 214, 249],
 'structure_id_path': [997, 8, 567, 623, 477, 485, 672]}
